In [1]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split

# all emotions on RAVDESS dataset
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "neutral",
    "happy"
}

In [2]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [18]:
def load_data(test_size=0.2):
    X, y = [], []
    try :
      for file in glob.glob("/content/drive/MyDrive/RAVDESS_speech/Actor_*/*.wav"):
          # get the base name of the audio file
          basename = os.path.basename(file)
          print(basename)
          # get the emotion label
          emotion = int2emotion[basename.split("-")[2]]
          # we allow only AVAILABLE_EMOTIONS we set
          if emotion not in AVAILABLE_EMOTIONS:
              continue
          # extract speech features
          features = extract_feature(file, mfcc=True, chroma=False, mel=False)
          # add to data
          X.append(features)
          l={'happy':0.0,'sad':1.0,'neutral':3.0,'angry':4.0}
          y.append(l[emotion])
    except :
         pass
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

In [19]:
X_train, X_test, y_train, y_test = load_data(test_size=0.25)

print("[+] Number of training samples:", X_train.shape)
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape)

03-01-02-01-01-02-05.wav
03-01-01-01-02-01-05.wav
03-01-02-01-01-01-05.wav
03-01-01-01-01-01-05.wav
03-01-01-01-01-02-05.wav
03-01-01-01-02-02-05.wav
03-01-02-01-02-01-05.wav
03-01-03-02-01-02-05.wav
03-01-02-02-01-02-05.wav
03-01-02-02-02-02-05.wav
03-01-02-02-01-01-05.wav
03-01-02-01-02-02-05.wav
03-01-03-02-02-01-05.wav
03-01-03-01-01-02-05.wav
03-01-03-01-02-01-05.wav
03-01-02-02-02-01-05.wav
03-01-03-02-01-01-05.wav
03-01-03-01-02-02-05.wav
03-01-03-01-01-01-05.wav
03-01-05-01-02-02-05.wav
03-01-04-01-02-02-05.wav
03-01-05-02-01-02-05.wav
03-01-05-02-02-02-05.wav
03-01-05-01-01-01-05.wav
03-01-05-02-01-01-05.wav
03-01-04-02-01-02-05.wav
03-01-04-02-02-01-05.wav
03-01-05-01-01-02-05.wav
03-01-04-02-02-02-05.wav
03-01-04-01-01-02-05.wav
03-01-05-02-02-01-05.wav
03-01-04-02-01-01-05.wav
03-01-05-01-02-01-05.wav
03-01-04-01-01-01-05.wav
03-01-03-02-02-02-05.wav
03-01-04-01-02-01-05.wav
03-01-07-01-01-02-05.wav
03-01-07-02-01-02-05.wav
03-01-07-01-01-01-05.wav
03-01-07-02-01-01-05.wav


In [20]:
import numpy as np
X_train = np.asarray(X_train)
y_train= np.asarray(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)

In [21]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((326, 40), (326,), (109, 40), (109,))

In [22]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)


In [23]:
x_traincnn.shape,x_testcnn.shape

((326, 40, 1), (109, 40, 1))

In [39]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers

model = Sequential()

model.add(Conv1D(128, 5,padding='same',input_shape=(40,1)))        #1
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))



model.add(Conv1D(128, 5,padding='same',))                           #2
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(8))                                                 #3
model.add(Activation('softmax'))
opt = optimizers.RMSprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [40]:
model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_14 (Conv1D)          (None, 40, 128)           768       
                                                                 
 activation_21 (Activation)  (None, 40, 128)           0         
                                                                 
 dropout_14 (Dropout)        (None, 40, 128)           0         
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 5, 128)           0         
 1D)                                                             
                                                                 
 conv1d_15 (Conv1D)          (None, 5, 128)            82048     
                                                                 
 activation_22 (Activation)  (None, 5, 128)            0         
                                                      

In [41]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [42]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=20, epochs=500, validation_data=(x_testcnn, y_test))


Epoch 1/500
17/17 [==============================] - 10s 27ms/step - loss: 14.2561 - accuracy: 0.1503 - val_loss: 2.4373 - val_accuracy: 0.3670
Epoch 2/500
17/17 [==============================] - 0s 8ms/step - loss: 5.8376 - accuracy: 0.2638 - val_loss: 1.4515 - val_accuracy: 0.3211
Epoch 3/500
17/17 [==============================] - 0s 7ms/step - loss: 6.2036 - accuracy: 0.2577 - val_loss: 2.0853 - val_accuracy: 0.2110
Epoch 4/500
17/17 [==============================] - 0s 7ms/step - loss: 5.3048 - accuracy: 0.2577 - val_loss: 1.8496 - val_accuracy: 0.2752
Epoch 5/500
17/17 [==============================] - 0s 7ms/step - loss: 5.0941 - accuracy: 0.3067 - val_loss: 1.5879 - val_accuracy: 0.3119
Epoch 6/500
17/17 [==============================] - 0s 7ms/step - loss: 5.2360 - accuracy: 0.3190 - val_loss: 2.5866 - val_accuracy: 0.3028
Epoch 7/500
17/17 [==============================] - 0s 7ms/step - loss: 5.2835 - accuracy: 0.3037 - val_loss: 2.3066 - val_accuracy: 0.3945
Epoch 8/50

In [43]:
em=['happy','sad','neutral','angry']


In [45]:
loss, acc = model.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

4/4 [==============================] - 0s 13ms/step - loss: 0.8800 - accuracy: 0.6147
Restored model, accuracy: 61.47%
